# Import required libraries

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras import metrics
from keras import regularizers
import models.nnmodels as nn
import utils.data as data

import gc
gc.enable()
seed = 0
np.random.seed(seed)
%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 1
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

Using TensorFlow backend.


# Load data for two classes classification


In [2]:
print 'Loading DEAP dataset...'
subject_data_folds, valence_folds, arousal_folds = data.load_deap(32, 2)
print 'DEAP dataset is loaded.\n'

print 5*'#' + ' Dataset representation for each subject ' + 5*'#'
print '\nThere are {} subjects in the dataset.\n'.format(len(subject_data_folds))
print '(Trail/Video, Channel, Readings) : ', subject_data_folds[0].shape
print '(Trail/Video, label (valence or arousal)) : ', arousal_folds[0].shape



Loading DEAP dataset...
DEAP dataset is loaded.

##### Dataset representation for each subject #####

There are 32 subjects in the dataset.

(Trail/Video, Channel, Readings) :  (40, 40, 8064)
(Trail/Video, label (valence or arousal)) :  (40, 2)


# Reduce the features dimention (8064D >> 101D)
## Then Normalize the features

In [3]:
# Reduce the features dimension
print 'Mapping features into Gaussian with mean zero and unit variance...'
reduced_data_folds = data.normalize_features(subject_data_folds, flag=True)
print 'Normalization done.'

Mapping features into Gaussian with mean zero and unit variance...
Normalization done.


# Train CNN model for two classes (low arousal / High arousal) classification

In [ ]:
num_exp = 40
num_classes = 2
batch_size = 50
epochs = 250
num_subjects = 32


accuracies= []

print '##### Two Classes Classification for arousal emotion Using Convolutional Neural Network #####\n'
   

for s in np.arange(0, num_subjects):
    
    start = time.time()
    X_val = reduced_data_folds[s]
    X_val = X_val.reshape(X_val.shape[0], 1, 40, 101)
    y_val = arousal_folds[s]
    
    X_remaining_folds = reduced_data_folds[:s] + reduced_data_folds[s+1:]
    y_remaining_folds = arousal_folds[:s] + arousal_folds[s+1:]
    
    X_train = np.concatenate(X_remaining_folds)
    X_train = X_train.reshape(X_train.shape[0], 1, 40, 101)
    
    y_train = np.concatenate(y_remaining_folds)
    
    
    key = 's'+str(s+1).zfill(2)+'_arousal(2classes)'
    
    
    # arousal classification model for two classes    
    model = nn.CNN(num_classes=num_classes, emotion='arousal')
    
    
    # checkpoint
    filepath="../saved_models/best_CNN_model_for_{}.h5".format(key)
    
    print 'Training model to test on subject {} ...\n'.format(str(s+1).zfill(2))
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                verbose=1, save_best_only=True, mode='max', period=20)
    
    callbacks_list = [checkpoint]
    
    
    hist = model.fit(X_train, y_train, 
             batch_size=batch_size,
             epochs=epochs,
             shuffle=True,
             verbose=0,
             validation_data=(X_val, y_val), callbacks=callbacks_list)

    
    
    model.load_weights(filepath)
    
    scores = model.evaluate(x=X_val, y=y_val)
    
    
    
    print '#### Result of CNN model for subject {} #####\n'.format(str(s+1).zfill(2))
    print "Loss: {} , Accuracy : {}%".format(scores[0], scores[1]*100)
    print 'it takes', start - time.time()
    

    accuracies.append(scores[1])
    # summarize history for accuracy
    plt.plot(hist.history['categorical_accuracy'])
    plt.plot(hist.history['val_categorical_accuracy'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' DNN model accuracy for arousal')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model loss for arousal')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    del model
 

   

##### Two Classes Classification for arousal emotion Using Convolutional Neural Network #####

Training model to test on subject 01 ...

Epoch 00020: val_categorical_accuracy improved from -inf to 0.62500, saving model to ../saved_models/best_CNN_model_for_s01_arousal(2classes).h5
Epoch 00040: val_categorical_accuracy did not improve
Epoch 00060: val_categorical_accuracy did not improve
Epoch 00080: val_categorical_accuracy did not improve
Epoch 00100: val_categorical_accuracy did not improve
Epoch 00120: val_categorical_accuracy did not improve
Epoch 00140: val_categorical_accuracy improved from 0.62500 to 0.65000, saving model to ../saved_models/best_CNN_model_for_s01_arousal(2classes).h5
Epoch 00160: val_categorical_accuracy did not improve
Epoch 00180: val_categorical_accuracy did not improve
Epoch 00200: val_categorical_accuracy did not improve
Epoch 00220: val_categorical_accuracy did not improve
Epoch 00240: val_categorical_accuracy did not improve


# Accuracies

In [8]:
print np.argmin(accuracies), np.argmax(accuracies)
print reduce(lambda x, y: x + y, accuracies) / len(accuracies)
accuracies

15 19
0.76171875


[0.72499999999999998,
 0.82499999999999996,
 0.72499999999999998,
 0.69999999999999996,
 0.69999999999999996,
 0.72499999999999998,
 0.72499999999999998,
 0.77500000000000002,
 0.77500000000000002,
 0.82499999999999996,
 0.65000000000000002,
 0.84999999999999998,
 0.77500000000000002,
 0.84999999999999998,
 0.84999999999999998,
 0.52500000000000002,
 0.80000000000000004,
 0.82499999999999996,
 0.82499999999999996,
 0.90000000000000002,
 0.75,
 0.82499999999999996,
 0.75,
 0.84999999999999998,
 0.75,
 0.67500000000000004,
 0.59999999999999998,
 0.80000000000000004,
 0.75,
 0.72499999999999998,
 0.80000000000000004,
 0.75]